In [2]:
# Imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Daten laden
# Laden der beiden CSV-Dateien
file_path_sun_data = 'data\\meteo_swiss_data.csv'
file_path_stn_data = 'data\\meteo_swiss_stn_data.csv'

sun_data = pd.read_csv(file_path_sun_data, delimiter=';')
stn_data = pd.read_csv(file_path_stn_data, delimiter=';')

# Join der beiden DataFrames anhand der 'stn'-Spalte
data = pd.merge(sun_data, stn_data, on='stn', how='inner')

# Daten konvertieren
data['time'] = pd.to_datetime(data['time'], format='%Y%m')
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month
data['sunhours'] = data['su2000m0'].replace('-', '0').astype(float)

# Daten filtern
data = data[(data['sunhours'] > 0) & (data['year'] != 2025) & (data['canton'] != 'LI')]
data['sunhours'] = data['sunhours'].astype(float)

monthly_data = data.groupby(['year', 'month'])['sunhours'].mean().unstack(level=0)

# Deutsche Monatsnamen
month_names = ['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']


# Berechnen des durchschnittlichen Sonnenstunden pro Monat über alle Jahre
average_sunhours = monthly_data.mean(axis=1)

# Berechnen des Medians der Sonnenstunden pro Monat über alle Jahre
median_sunhours = monthly_data.median(axis=1)


# Interaktives Liniendiagramm mit Plotly
fig = go.Figure()

# Linien für jedes Jahr hinzufügen
for year in monthly_data.columns:
    fig.add_trace(go.Scatter(x=monthly_data.index, y=monthly_data[year], mode='lines+markers', name=str(year)))

# Durchschnittslinie hinzufügen
fig.add_trace(go.Scatter(x=monthly_data.index, y=average_sunhours, mode='lines', name='Durchschnitt', line=dict(color='black', dash='dash')))

# Medianlinie hinzufügen
fig.add_trace(go.Scatter(x=monthly_data.index, y=median_sunhours, mode='lines', name='Median', line=dict(color='red', dash='dot')))

# Layout anpassen
fig.update_layout(
    title='Durchschnittliche Sonnenstunden pro Monat',
    xaxis_title='Monat',
    yaxis_title='Durchschnittliche Sonnenstunden',
    xaxis=dict(tickmode='array', tickvals=list(range(1, 13)), ticktext=month_names),
    legend_title='Jahr',
    template='plotly_white'
)

fig.show()

fig.write_html('docs\\assets\\diagramme\\sunhours_per_month.html')

# Interaktiver Boxplot mit Plotly
fig_box = px.box(data, x='month', y='sunhours', points='all', labels={'month': 'Monat', 'sunhours': 'Sonnenstunden'})
fig_box.update_layout(
    title='Verteilung der Sonnenstunden pro Monat',
    xaxis=dict(tickmode='array', tickvals=list(range(1, 13)), ticktext=month_names),
    template='plotly_white'
)

fig_box.show()
